In [13]:
# args

# persona_val_path = "opinions_qa/persona_val/American_Trends_Panel_W34/date0828_personas_American_Trends_Panel_W34_testonly_haiku.json"
persona_val_path = "opinions_qa/persona_val/American_Trends_Panel_W34/date0831_personas_full_haiku_known_test.json"


In [14]:
import os
os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

from transformers import AutoTokenizer, AutoModel
import json



In [15]:
with open(persona_val_path, 'r') as f:
    p_vals = json.load(f)
print(len(list(p_vals.keys())))

2537


In [16]:
import random
import pandas as pd
from src.utils import get_file_from_s3
random.seed(42)

file_key = "American_Trends_Panel_W34"
resp_df = pd.read_csv(get_file_from_s3(f"human_resp/{file_key}/responses.csv"))
test_user_idx = random.choices(range(len(resp_df)), k=int(len(resp_df)*0.1))
test_resp_df = resp_df.iloc[test_user_idx]
test_len = int(len(resp_df)*0.1)

test_user_list = list(p_vals.keys())[:test_len]
train_user_list = list(p_vals.keys())[test_len:]

print('test:', len(test_user_list))
print('train:', len(train_user_list))

test: 253
train: 2284


In [17]:
def get_user_pval_df(pvals, user_list):
    records = []
    for user in user_list:
        personas = pvals[user]
        entry = {'user': user}
        for p in personas:
            entry[p['name']] = p['inferred_value']
        records.append(entry)

    df = pd.DataFrame(records)
    return df

In [18]:
df = get_user_pval_df(p_vals, train_user_list)
df[:3]

,user,Safety vs. Timeliness Prioritization,Health and Nutrition Values,Personal Values on Gene Editing,Concern for Social Justice,Worldview and Belief System,Openness to Food Innovation,Trust in Science and Technology,Lifestyle Priorities,Core Personal Values,...,Attitude Towards Science,Risk Perception,Technological Outlook,Personal Health Concerns,Technological Awareness,Dietary Habits,Healthcare System Familiarity,Information Sources on GMOs,Access to Healthcare,Personal Experiences with Scientific and Technological Advancements
0,13,prioritizes safety over speed,balanced values,values caution and preservation,low concern,conservative,resistant to innovation,low trust,prioritizes other lifestyle factors,prioritizes individual freedom,...,negative,low risk perception,pessimistic,does not have personal health concerns,low awareness,consistently unhealthy,low familiarity,non-scientific sources,low access,negative experiences
1,14,prioritizes safety over speed,balanced values,values caution and preservation,moderate concern,moderate,resistant to innovation,low trust,balanced priorities,prioritizes safety,...,negative,low risk perception,pessimistic,does not have personal health concerns,low awareness,balanced,moderate familiarity,trusts non-scientific sources,low access,negative experiences
2,15,prioritizes safety over speed,balanced values,values caution and preservation,high concern,moderate,open to innovation,high trust,prioritizes health and nutrition,prioritizes individual freedom,...,positive,low risk perception,optimistic,has personal health concerns,low awareness,balanced,moderate familiarity,trusts scientific sources,low access,positive experiences


In [19]:
from collections import Counter

skew_cnt = 0
skew_personas = []
for col in df.columns:
    if col == 'user': continue
    cnt = Counter(df[col].values)
    if len(cnt.keys()) < 2: 
        skew_cnt += 1
        skew_personas.append(col) 
        print(col)
        print(Counter(df[col].values))
        print()
        continue
    first, second = cnt.most_common(2)
    if first[1] / second[1] > 50:
        skew_cnt += 1
        skew_personas.append(col) 
        print(col)
        print(Counter(df[col].values))
        print()
print(skew_cnt / len(df.columns))


Health and Nutrition Values
Counter({'balanced values': 2225, 'values convenience and indulgence': 40, 'values health and wellness': 19})

Personal Values on Gene Editing
Counter({'values caution and preservation': 2261, 'values progress and innovation': 23})

Ethical Orientation
Counter({'low ethical concern': 2280, 'high ethical concern': 4})

Attitudes Towards Genetic Engineering
Counter({'skeptical': 2274, 'optimistic': 10})

Food Motivations
Counter({'balanced': 2283, 'pleasure-focused': 1})

0.1282051282051282


In [20]:
su = test_user_list[0]

def get_user_persona_repr(personas):
    return '; '.join(f"{p['name']}: {p['inferred_value']}" for p in personas if p['name'] not in skew_personas)

get_user_persona_repr(p_vals[su])

'Safety vs. Timeliness Prioritization: prioritizes safety over speed; Concern for Social Justice: moderate concern; Worldview and Belief System: moderate; Openness to Food Innovation: resistant to innovation; Trust in Science and Technology: low trust; Lifestyle Priorities: balanced priorities; Core Personal Values: prioritizes individual freedom; Ethical Concerns about Gene Editing: high ethical concerns; Personal Values Towards Genetically Modified Foods: prioritizes health and safety; Trust in Healthcare: high trust; Environmental Consciousness: low environmental consciousness; Perceived Risk of Genetically Modified Foods: low risk perception; Self-Discipline: moderate; Perception of Technological Equity: indifferent to inequality; Perception of Healthcare System Efficiency: does not see it as a problem; Health Consciousness: moderate health consciousness; Emotional Relationship with Food: neutral; Food Preferences: indifferent to food type; Scientific Literacy: moderate; Nutrition 

In [21]:
tok = AutoTokenizer.from_pretrained('nvidia/NV-Embed-v2')

len(tok.tokenize(get_user_persona_repr(p_vals[su])))

364

In [22]:


import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import time

# Each query needs to be accompanied by an corresponding instruction describing the task.
task_name_to_instruct = {"retrieve_user_personas": "Given a user persona description, retrieve similar user presona descriptions",}

query_prefix = "Instruct: "+task_name_to_instruct["retrieve_user_personas"]+"\nQuery: "
queries = [
    get_user_persona_repr(p_vals[user]) for user in test_user_list
]

# No instruction needed for retrieval passages
passage_prefix = ""
passages = [
    get_user_persona_repr(p_vals[user]) for user in train_user_list[:200]
]

# load model with tokenizer
model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True, device_map='auto')

# get the embeddings
max_length = 4096
tic = time.time()
# query_embeddings = model.encode(queries, instruction=query_prefix, max_length=max_length)
# passage_embeddings = model.encode(passages, instruction=passage_prefix, max_length=max_length)
batch_size=2
query_embeddings = model._do_encode(queries, batch_size=batch_size, instruction=query_prefix, max_length=max_length, num_workers=32, return_numpy=True)
toc = time.time()
print('Test encoding time:', toc-tic)

passage_embeddings = model._do_encode(passages, batch_size=batch_size, instruction=passage_prefix, max_length=max_length, num_workers=32, return_numpy=True)
toc = time.time()
print('Total encoding time:', toc-tic)

# normalize embeddings
# query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
# passage_embeddings = F.normalize(passage_embeddings, p=2, dim=1)
query_embeddings = F.normalize(torch.tensor(query_embeddings), p=2, dim=1)
passage_embeddings = F.normalize(torch.tensor(passage_embeddings), p=2, dim=1)

scores = (query_embeddings @ passage_embeddings.T) * 100
# print(scores.tolist())
# [[87.42692565917969, 0.462837278842926], [0.9652643203735352, 86.0372314453125]]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

encoding:   0%|          | 0/127 [00:01<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 

In [ ]:
scores

tensor([[78.6956, 81.3270, 79.4460,  ..., 84.1884, 83.9234, 80.1651],
        [79.3121, 82.8288, 81.6716,  ..., 83.2478, 83.0006, 82.4287],
        [80.0940, 78.5115, 76.5741,  ..., 81.8465, 82.3200, 77.8372],
        ...,
        [75.2231, 81.9015, 80.3835,  ..., 79.8307, 79.5554, 83.6244],
        [77.8370, 83.3969, 80.9174,  ..., 82.8269, 82.7837, 82.1167],
        [71.1475, 79.2757, 82.5588,  ..., 78.2288, 77.2570, 79.6065]])

In [ ]:
import numpy as np

def get_top_k_similar_users(scores_matrix, query_user_names, corpus_user_names, k):
    top_k_similar_users = {}
    scores_matrix = scores_matrix.detach().cpu().numpy()

    for i, curr_scores in enumerate(scores_matrix):
        # Get the indices of the top k scores
        top_k_indices = np.argsort(curr_scores)[-k:][::-1]
        
        # Map indices to corresponding user names in the corpus_user_names list
        top_k_users = [corpus_user_names[idx] for idx in top_k_indices]
        
        # Map the query user name to the list of top k similar users
        top_k_similar_users[query_user_names[i]] = top_k_users

    return top_k_similar_users

k = 20  # Number of top similar users to retrieve
result = get_top_k_similar_users(scores, test_user_list, train_user_list, k)
print(result)


{'1622': ['92', '195', '125', '135', '233', '221', '17', '133', '234', '58', '19', '162', '112', '124', '142', '157', '197', '215', '201', '96'], '63': ['101', '40', '160', '71', '92', '189', '221', '168', '125', '17', '58', '231', '195', '229', '120', '152', '233', '62', '133', '50'], '697': ['122', '47', '19', '201', '53', '172', '195', '83', '133', '82', '135', '234', '162', '125', '197', '96', '33', '86', '233', '183'], '566': ['120', '152', '223', '177', '100', '35', '205', '50', '103', '199', '73', '109', '92', '149', '42', '34', '43', '40', '76', '185'], '1868': ['135', '195', '92', '125', '112', '221', '58', '233', '17', '234', '162', '24', '96', '133', '183', '101', '142', '44', '124', '215'], '1716': ['162', '135', '105', '86', '198', '41', '172', '44', '125', '134', '70', '126', '13', '83', '19', '178', '77', '82', '209', '54'], '2263': ['151', '210', '155', '189', '108', '231', '74', '236', '141', '140', '216', '27', '20', '65', '64', '87', '190', '164', '208', '84'], '220'

In [ ]:
import os
os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

from collections import Counter 

# path = "opinions_qa/review/American_Trends_Panel_W34/date0901_v16_known_test_rag_2000_40_withname_American_Trends_Panel_W34_haikupred_prompt3_history_demo_persona_run1"
# path = "opinions_qa/review/American_Trends_Panel_W34/date0901_v16_known_test_rag_500_40_withname_American_Trends_Panel_W34_haikupred_prompt3_history_demo_persona_run1"
path = "opinions_qa/review/American_Trends_Panel_W34/date0901_v16_known_test_rag_train2000_top40_skew10_usedemoTrue_withname_American_Trends_Panel_W34_haikupred_prompt3_history_run1"

# Usage example
start_marker = "# Similar users' answers"
end_marker = "# Your Prediction of the Participant's Response (answer with given options verbatim, no formatting, no punctuations or linebreaks at the end, don't explain)"


def extract_text_between_markers(file_path, start_marker, end_marker):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    gold = lines[-1][len("Gold: "):]

    # Initialize variables to track whether we are within the markers
    capturing = False
    extracted_lines = []

    for line in lines:
        if start_marker in line:
            capturing = True
            continue  # Skip the start marker line itself
        elif end_marker in line:
            capturing = False
            break  # Stop once we reach the end marker line

        if capturing:
            extracted_lines.append(line.strip())  # Strip any surrounding whitespace

    return gold, "\n".join(extracted_lines)

correct, total = 0, 0
for filename in os.listdir(path):
    file_path = f"{path}/{filename}"
    gold, extracted_text = extract_text_between_markers(file_path, start_marker, end_marker)
    most = Counter(extracted_text.split("\n")).most_common(1)[0][0]
    if most == gold: correct += 1
    total += 1
print(correct/total)

0.5726141078838174


'A big problem\nA big problem\nA small problem\nA small problem\nA big problem\nNot a problem\nNot a problem\nA big problem\nA small problem\nA small problem\nA small problem\nA big problem\nA small problem\nA big problem\nA small problem\nA big problem\nA big problem\nA small problem\nA big problem\nA big problem\nA big problem\nA small problem\nA small problem\nA small problem\nA big problem\nA big problem\nA big problem\nA small problem\nA small problem\nA small problem\nA big problem\nNot a problem\nA big problem\nNot a problem\nA big problem\nA small problem\nA small problem\nA big problem\nA small problem\nA small problem\n'

# Hyperparameter Tuning

In [ ]:
from transformers import AutoTokenizer, AutoModel
import json
import random
import pandas as pd
from src.utils import get_file_from_s3
random.seed(42)


def main(args):


    with open(args.persona_val_path, 'r') as f:
        p_vals = json.load(f)
    print("Number of users:", len(list(p_vals.keys())))


    resp_df = pd.read_csv(get_file_from_s3(f"human_resp/{args.survey_name}/responses.csv"))
    test_user_idx = random.choices(range(len(resp_df)), k=int(len(resp_df)*0.1))
    test_resp_df = resp_df.iloc[test_user_idx]
    test_len = int(len(resp_df)*0.1)

    test_user_list = list(p_vals.keys())[:test_len]
    train_user_list = list(p_vals.keys())[test_len:]

    print('test users:', len(test_user_list))
    print('train users:', len(train_user_list))

    if args.use_demo:
        # get demo mapping
        meta_df = pd.read_csv(get_file_from_s3(f"human_resp/{args.survey_name}/metadata.csv"))
        meta_keys = list(meta_df['key'])


    def get_user_pval_df(pvals, user_list):
        
        records = []
        for user in user_list:
            personas = pvals[user]
            entry = {'user': user}
            for p in personas:
                entry[p['name']] = p['inferred_value']

            records.append(entry)

        df = pd.DataFrame(records)
        return df


    df = get_user_pval_df(p_vals, train_user_list)

    from collections import Counter

    skew_cnt = 0
    skew_personas = []
    for col in df.columns:
        if col == 'user': continue
        cnt = Counter(df[col].values)
        if len(cnt.keys()) < 2: 
            skew_cnt += 1
            skew_personas.append(col) 
            print(col)
            print(Counter(df[col].values))
            print()
            continue
        first, second = cnt.most_common(2)
        if first[1] / second[1] > args.skew:
            skew_cnt += 1
            skew_personas.append(col) 
            print(col)
            print(Counter(df[col].values))
            print()
    print(skew_cnt / len(df.columns))


    su = test_user_list[0]

    def get_user_persona_repr(user, use_demo):
        personas = p_vals[user]
        if use_demo:
            row = resp_df.iloc[int(user)]
            # get demo
            demo = row[meta_keys].to_dict()
            key_mappings = {
                'CREGION': 'Region Where The Participant Lives',
                'AGE': 'Age Range Of The Participant',
                'SEX': 'Gender Of The Participant',
                'EDUCATION': 'Educational Attainment Of The Participant',
                'CITIZEN': 'Citizenship Status Of The Participant',
                'MARITAL': 'Marital Status Of The Participant',
                'RELIG': 'Religious Affiliation Of The Participant',
                'RELIGATTEND': 'Frequency Of Religious Service Attendance',
                'POLPARTY': 'Political Party Affiliation',
                'INCOME': 'Annual Income Range',
                'POLIDEOLOGY': 'Political Ideology',
                'RACE': 'Racial Background'
            }
            demo = {key_mappings[k]: v for k, v in demo.items()}
            for k, v in demo.items():
                personas.append({
                    'name': k,
                    'inferred_value': v
                })

        # return '; '.join(f"{p['name']}: {p['inferred_value']}" for p in personas)
        return '; '.join(f"{p['name']}: {p['inferred_value']}" for p in personas if p['name'] not in skew_personas)

    print(get_user_persona_repr(su, args.use_demo))



    import torch
    import torch.nn.functional as F
    from transformers import AutoTokenizer, AutoModel
    import time

    # Each query needs to be accompanied by an corresponding instruction describing the task.
    task_name_to_instruct = {"retrieve_user_personas": "Given a user persona description, retrieve similar user presona descriptions",}

    query_prefix = "Instruct: "+task_name_to_instruct["retrieve_user_personas"]+"\nQuery: "
    queries = [
        get_user_persona_repr(user, args.use_demo) for user in test_user_list
    ]

    # No instruction needed for retrieval passages
    passage_prefix = ""
    passages = [
        get_user_persona_repr(user, args.use_demo) for user in train_user_list[:args.train_size]
    ]

    # load model with tokenizer
    model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True, device_map='auto')

    # get the embeddings
    max_length = 4096
    tic = time.time()
    # query_embeddings = model.encode(queries, instruction=query_prefix, max_length=max_length)
    # passage_embeddings = model.encode(passages, instruction=passage_prefix, max_length=max_length)
    batch_size=2
    query_embeddings = model._do_encode(queries, batch_size=batch_size, instruction=query_prefix, max_length=max_length, num_workers=32, return_numpy=True)
    toc = time.time()
    print('Test encoding time:', toc-tic)

    passage_embeddings = model._do_encode(passages, batch_size=batch_size, instruction=passage_prefix, max_length=max_length, num_workers=32, return_numpy=True)
    toc = time.time()
    print('Total encoding time:', toc-tic)
    query_embeddings = F.normalize(torch.tensor(query_embeddings), p=2, dim=1)
    passage_embeddings = F.normalize(torch.tensor(passage_embeddings), p=2, dim=1)

    scores = (query_embeddings @ passage_embeddings.T) * 100

    import numpy as np

    def get_top_k_similar_users(scores_matrix, query_user_names, corpus_user_names, k):
        top_k_similar_users = {}
        scores_matrix = scores_matrix.detach().cpu().numpy()

        for i, curr_scores in enumerate(scores_matrix):
            # Get the indices of the top k scores
            top_k_indices = np.argsort(curr_scores)[-k:][::-1]
            
            # Map indices to corresponding user names in the corpus_user_names list
            top_k_users = [corpus_user_names[idx] for idx in top_k_indices]
            
            # Map the query user name to the list of top k similar users
            top_k_similar_users[query_user_names[i]] = top_k_users

        return top_k_similar_users


    result = get_top_k_similar_users(scores, test_user_list, train_user_list, args.top_k)

    return result


if __name__ == '__main__':

    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--survey_name', type=str)
    parser.add_argument('--persona_val_path', type=str)
    parser.add_argument('--user_mapping_filepath', type=str)
    parser.add_argument('--train_size', type=int)
    parser.add_argument('--top_k', type=int)
    parser.add_argument('--skew', type=int)
    parser.add_argument('--use_demo', action='store_true')
    args = parser.parse_args()
    main(args)

In [1]:
import os
import json

os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [2]:
with open('hp_grid_search_v2.json') as f:
    search = json.load(f)

In [3]:
import pandas as pd
import tabulate

df = pd.DataFrame(search)

df.sort_values(by='result', ascending=False)

subdf = df[['setting', 'top_k', 'skew_thres', 'result']].sort_values(by='result', ascending=False)
print(tabulate.tabulate(subdf, headers='keys', tablefmt='psql'))

+----+--------------+---------+--------------+----------+
|    | setting      |   top_k |   skew_thres |   result |
|----+--------------+---------+--------------+----------|
| 15 | demo_persona |     250 |            5 | 0.590909 |
| 50 | persona_only |     200 |            3 | 0.585639 |
| 45 | persona_only |     100 |            3 | 0.58498  |
| 14 | demo_persona |     200 |            5 | 0.58498  |
| 47 | persona_only |     140 |            3 | 0.584321 |
| 13 | demo_persona |     180 |            5 | 0.582345 |
| 22 | demo_persona |     180 |            3 | 0.581686 |
| 10 | demo_persona |     120 |            5 | 0.581028 |
| 21 | demo_persona |     160 |            3 | 0.580369 |
| 16 | demo_persona |     300 |            5 | 0.57971  |
| 25 | demo_persona |     300 |            3 | 0.57971  |
|  7 | demo_persona |     300 |           10 | 0.57971  |
|  4 | demo_persona |     180 |           10 | 0.57971  |
|  3 | demo_persona |     160 |           10 | 0.57971  |
| 11 | demo_pe

In [64]:
setting_mapping = {
    "demo_persona": "querydp",
    "demo_only": "querydonly",
    "persona_only": "queryponly",
}

In [66]:
row = df.iloc[23]
row

setting                                                      demo_persona
skew_thres                                                              3
top_k                                                                  40
result                                                            0.57971
similar_user_mapping    {'1622': ['77', '1314', '1283', '70', '1426', ...
Name: 23, dtype: object

In [67]:
date = '0902'

mapping_name = f"date{date}_personas_full_haiku_known_test_{setting_mapping[row['setting']]}_trainAll_top{row['top_k']}_skew{row['skew_thres']}_withname.json"

with open(f'opinions_qa/similar_users/American_Trends_Panel_W34/{mapping_name}', 'w') as f:
    json.dump(row['similar_user_mapping'], f, indent=4)

In [70]:
row = df.iloc[36]
row

setting                                                      persona_only
skew_thres                                                             10
top_k                                                                  70
result                                                           0.583004
similar_user_mapping    {'1622': ['1914', '201', '2272', '1921', '908'...
Name: 36, dtype: object

In [71]:
date = '0902'

mapping_name = f"date{date}_personas_full_haiku_known_test_{setting_mapping[row['setting']]}_trainAll_top{row['top_k']}_skew{row['skew_thres']}_withname.json"

with open(f'opinions_qa/similar_users/American_Trends_Panel_W34/{mapping_name}', 'w') as f:
    json.dump(row['similar_user_mapping'], f, indent=4)